# Predictions for Customer Lifetime Value 

## Problem overview:

This scenario focuses on predictions related to CLV (Customer Lifetime Value) where the goal is to predict the future value a customer brings to the business. For any marketing department, the CLV is a key metric that helps understand the revenue attached to the future relationship with the customer. In general, the CLV is used to optimize marketing spends while maximizing returns. While there are many ways to model the CLV, in this scenario we will use a metric related to future customer spend in a given time range (e.g. a quarter). Predicting the future spend relies on the known transaction history of customers. One of the most widely used models to capture customer behavior is RFM (Recency, Frequency, Monetary). RFM features are derived from the transactional history of the customers and then used as inputs for the machine learning models that predict future spend.

This scenario details the development of a machine learning future customer spend prediction model. The model is trained on a public dataset containing transactions from an online retailer. The dataset contains the history of purchased items over 12 calendar months.

## Solution overview:

We will use the [Automated Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-auto-train) (autoML) capabilities of the [Azure Machine Learning service](https://docs.microsoft.com/en-us/azure/machine-learning/service/overview-what-is-azure-ml) to quickly train a model that can predict future customer spend. We will model our problem as a **Regression** problem where the goal of the trained model is to predict a numerical value (in our case, the future customer spend). The automML capabilities enable us to evaluate different algorithms and hyperparameters to get the best trained model for the problem with minimum effort. The approach used in this example cand be extended to various use cases that revolve around the need to predict numerical values related to customer behavior (and not only).


This notebook is organized into the following sections:

1. Basic setup

2. Data prep

3. Model training

4. Explore the results and evaluate the best model

5. Model Explainability: which features matter for the predictions?

## Section 1. Basic setup

Make the necessary namespace and class imports.

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import calendar as cal
import logging
import warnings
# Squash warning messages for cleaner output in the notebook
warnings.showwarning = lambda *args, **kwargs: None

from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import azureml.core
from azureml.core.workspace import Workspace
from azureml.core.experiment import Experiment
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.automlexplainer import explain_model

Before starting this step, you need to create an Azure Machine Learning service workspace ([instructions](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace)).

Let's get started by creating an experiment in your Azure Machine Learning workspace. An experiment is a named object in a workspace, which is used to do model training.

In [ ]:
subscription_id = "<subscription id goes here>"
resource_group = "<resource group goes here>"
workspace_name = "<workspace name goes here>"

In [ ]:
ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

In [ ]:
# choose a name for the run history container in the workspace
experiment_name = 'CLVFutureSpend'

# project folder
project_folder = './sample_projects/automl-clvfuturespendprediction'

experiment=Experiment(ws, experiment_name)

output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(data = output, index = ['']).T

## Section 2. Data prep

OnlineRetail.csv contains the customer purchase history between December 1st, 2010 and December 9th, 2011.

In [ ]:
data = pd.read_csv("https://quickstartsws9073123377.blob.core.windows.net/amlnotebooktutorials/customer-lifetime-value/OnlineRetail.csv", encoding='cp1252', parse_dates=['InvoiceDate'])

Clean the dataset by removing CustomerID missing values and convert column to integer.

In [ ]:
data = data[np.isnan(data.CustomerID) == False]
data['CustomerID'] = pd.to_numeric(data['CustomerID'], downcast='integer')

## 2.1 Inspect and clean data

Display the first few rows of the data and view some plots to help you understand the dynamics within the dataset.

In [ ]:
data.head(20)

In [ ]:
print('Sales data is available between {} and {}'.format(data.InvoiceDate.min(), data.InvoiceDate.max()))

We will keep data for exactly 12 months. The problem we want to solve is: given R, F, and M values for 6 months, predict the value of M for the next 6 months.

In [ ]:
data = data[data['InvoiceDate'] < dt.date(2011, 12, 1)]

We are not taking into account returns, so all entries with negative quantities will be discarded.

In [ ]:
clean_data = data[data['Quantity'] > 0].copy()
clean_data = clean_data[clean_data['UnitPrice'] > 0]
clean_data.describe()

The statistical analysis shows that there are some quantities that are abnormaly high. Let's plot the distribution of quantity and identify an upper limit for quantity (we will discard all entries with quantities exceeding the limit as outliers).

**Note**: We are using this over-simplified approach for outliers handling as we aim to keep the focus on the overall flow of the scenario. In production-grade solutions, a much more detailed and complex analysis must be performed.

In [ ]:
plt.figure(figsize=(20, 3))
plt.hist(clean_data['Quantity'], bins=20)
plt.show()

In [ ]:
clean_data = clean_data[clean_data['Quantity'] < 30]
clean_data.describe()

The statistical analysis shows that there are some unit prices that are abnormaly high. Let's plot the distribution of unit price values and identify an upper limit for unit price (we will discard all entries with unit prices exceeding the limit as outliers).

**Note**: We are using this over-simplified approach for outliers handling as we aim to keep the focus on the overall flow of the scenario. In production-grade solutions, a much more detailed and complex analysis must be performed.

In [ ]:
plt.figure(figsize=(20, 3))
plt.hist(clean_data['UnitPrice'], bins=20)
plt.show()

In [ ]:
clean_data = clean_data[clean_data['UnitPrice'] < 30]
clean_data.describe()

## 2.2 Engineer new features - Recency, Frequency, and Monetary

RFM (Recency, Frequency, Monetary) features will be calculated using the calendar month as a time period.

Let's start by creating new columns for month and year.

In [ ]:
def get_period(x):
    if x.date() < dt.date(2011, 6, 1):
        return 1
    else:
        return 2

clean_data['HalfYear'] = clean_data.InvoiceDate.apply(get_period)
clean_data['NextHalfYear'] = clean_data['HalfYear'] + 1
clean_data['Amount'] = clean_data.apply(lambda x: x.Quantity * x.UnitPrice, axis=1)
#check dataframe structure
clean_data.head(20)

Aggregate sales information by CustomerID, Year, and Month. Sum by amount, max by invoice date and count by individual invoice - all these will be needed to calculate the RFM features.

In [ ]:
aggregations = {
    'Amount':'sum', #sum of all invoices monetary value
    'InvoiceDate':'max', #last invoice in quarter
    'InvoiceNo': 'nunique' #count of invoices
}
#group by customerid and quarter value
rfm_data = clean_data.groupby(['CustomerID','HalfYear', 'NextHalfYear'], as_index=False).agg(aggregations)

Calculate the Recency feature for each customer as the number of days between the day of the last invoice in the quarter and the last day of the quarter. Note that the last invoice date was calculated in the previous step using the *max* function applied to the *InvoiceDate* property.

In [ ]:
def recency(x):
    #last = cal.monthrange(x.InvoiceDate.year, x.InvoiceDate.month)[1] #last day of last month in quarter
    #return (dt.date(x.InvoiceDate.year, x.InvoiceDate.month, last) - x.InvoiceDate.date()).days
    ref_date = ( dt.date(2011, 5, 31) if x.HalfYear == 1 else dt.date(2011, 11, 30) )
    return (ref_date - x.InvoiceDate.date()).days

rfm_data['R'] = rfm_data.apply(recency, axis=1)
rfm_data.head(20)

The Frequency and Monetary features are already calculated, we just need to give them proper names.

In [ ]:
rfm_data.rename(columns={
                   'InvoiceNo': 'F',
                   'Amount': 'M'}, inplace=True)
rfm_data = rfm_data[['CustomerID', 'HalfYear', 'NextHalfYear', 'R', 'F', 'M']]
rfm_data.head(20)

To calculate the next period's M (Monetary) value for each RFM combination, we need to join ```rfm_data``` with itself, using the ```NextHalfYear``` and ```HalfYear``` columns as keys.

The column is named ```M_Next``` and is the target value for prediction. In other words, our trained model will be able to predict ```M_Next``` given the values for R, F, and M that correspond to the current half year.

In [ ]:
rfm_data_final = pd.merge(rfm_data, rfm_data, left_on=['CustomerID', 'NextHalfYear'], right_on=['CustomerID', 'HalfYear'], how='left', suffixes=['', '_Next'])

Remove all entries that do not have values for the next month (to avoid biasing the model).

In [ ]:
rfm_data_final = rfm_data_final[['R', 'F', 'M', 'M_Next']]
rfm_data_final = rfm_data_final[np.isnan(rfm_data_final['M_Next']) == False]
rfm_data_final.head(20)

Let's look at the distributions for ```M``` and ```M_Next```.

In [ ]:
fig = plt.figure(figsize=(20,6))

ax1 = fig.add_subplot(211)
ax2 = fig.add_subplot(212)
ax1.hist(rfm_data_final['M'], bins=100)
ax2.hist(rfm_data_final['M_Next'], bins=100)
fig.show()

In [ ]:
rfm_data_final.describe()

## 2.3 Split the data into train and test sets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    rfm_data_final[['R', 'F', 'M']], rfm_data_final['M_Next'], test_size=0.1, random_state=100)

print('Number of training cases: {}'.format(len(X_train)))
print('Number of test cases: {}'.format(len(X_test)))

## Section 3. Model training

In this section you will configure the [automated ML](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-automated-ml) feature of the Azure Machine Learning service. Using the configuration you will then run an automated ML experiment which explores various algorithms and hyperparameter values to generate machine learning models. Finally, the best model will be selected. The training jobs run on local compute resources (provided and managed by Azure Notebooks).

The significant advantage of automated ML is the acceleration of a data scientist's work (as it does a significant portion of the exploration work). Besides that, automated ML exposes rich data resulting from experiment runs which enables control, transparency, and, most importantly, visibility on what is happening behind the scenes.

The configuration data required by automated ML contains information about the experiment itself as well as the training data used to train the models. Below is an example of the most important components of the configuration data:

Property | Description
--- | ---
task | regression
primary_metric | Metric that you want to optimize.<br>Forecasting supports the following primary metrics:<br>spearman_correlation<br>normalized_root_mean_squared_error<br>r2_score<br>normalized_mean_absolute_error
iterations | Number of iterations. In each individual iteration, automated ML trains one pipeline (algorithm and hyperparameters) on the given data.
iteration_timeout_minutes | The maximum number of minutes for each individual iteration
X | Training data in the form [n_samples, n_features]
y | Target values in the form [n_samples]
n_cross_validations | Number of [cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) splits.
path | Relative path to the project folder. Automated ML stores configuration files for the experiment under this folder. You can specify a new empty folder.


### 3.1 Automated ML configuration

Automated ML provides several options to configure the experiment runs, giving you flexibility and control. For example, the primary_metric setting specifies which metrich should automated ML use to optimize the machine learning model being built. There are multiple primary metrics available, and in our problem we will use NRMSE (Normalized Root Mean Squared Error).

Notice that we've set the task to regression and we are also specifying the training data set (X_train and y_train). We need to do this because training is performed locally. When training in performed remotely (e.g. on AML compute resources) you will need to provide a script that contains code to get the data instead of the data itself.

**Note**: The training time will be around 15 minutes with ```iterations``` set to 10 (meaning that Automated ML will train 10 different models using combinations of algorithms and hyperparameters). To reduce the training time, lower the ```iterations``` (to 5, for example) and the ```n_cross_validations``` (to 3, for example).

In [ ]:
automl_config = AutoMLConfig(task = 'regression',
                             debug_log = 'automl_clvfuturespend_errors.log',
                             primary_metric= 'normalized_root_mean_squared_error',
                             iterations = 10,
                             iteration_timeout_minutes = 5,
                             X = X_train,
                             y = y_train,
                             n_cross_validations = 5,
                             path=project_folder,
                             verbosity = logging.INFO)

### 3.2 Train your models on local compute

When you call the submit method on the experiment object and pass the AutoMLConfig object, automated ML will generate a number of machine learning models equal to **iterations** (10 in our case). Depending on the input data and the number of iterations the time required to complete the traning can range from a few minutes to hours (or even more). Once execution starts, you will see status messages being print out to the console.

In [ ]:
local_run = experiment.submit(automl_config, show_output=True)

### 3.3 Monitor training

To see the detailed results of the run, you can use a special widget (see the cell below).

**Note**: When you want to start the run and continue to execute your code you need to specify ```show_output=False``` when calling ```experiment.submit()```.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(local_run).show()

## Section 4. Explore the results and test the best model

## 4.1 Retrieve all child runs

Each individual model is trained in the context of a child run having **local_run** as its parent. You can get the list of all child runs and their logged metrics.

In [ ]:
children = list(local_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
rundata

## 4.2 Retrieve the best fitted model

The **get_output** method enables you to retrieve the best child run and the associated fitted model.

In [ ]:
best_run, fitted_model = local_run.get_output()
fitted_model.steps

## 4.3 Test the best fitted model

We will now use the best model trained by automated ML to make predictions on the test data. Finally, the RMSE value will be calculated for the test data.

**Important**: The results are acceptable, but not great. There are several reasons for this, among which are the limited number of customers and the limited time interval taken into consideration. keep in mind that our main goal in this tutorial is provide an overview of using Azure Machine Learning service to implement a practical solution to the problem of predicting the customer value based on a simple RFM data model. In production-grade scenarios, you will need to take into account other customer properties in addition to (or instead of) R, F, and M. Also, to accomodate for specific patterns like seasonality, several years worth of data must also be used to train the models. Finally, you will probably want test a wider range of algorithms and hyperparameter settings.

In [ ]:
y_pred = fitted_model.predict(X_test)

X_eval = X_test.copy()
X_eval['M_next'] = y_test.values
X_eval['M_next_pred'] = y_pred

rmse_test = ((X_eval.M_next - X_eval.M_next_pred) ** 2).mean() ** .5
print('RMSE for test data is {}'.format(rmse_test))

## Section 5. Model Explainability: Which features matter for the predictions?

Now that we have our trained model, we'd like to understand aspects related to its inner workings. One of the most important questions is "Which feature matters the most in calculating the predictions?".

To get the answer to the question, let's retrieve the explanation for our model.

In [ ]:
shap_values, expected_values, overall_summary, overall_imp, per_class_summary, per_class_imp = explain_model(fitted_model, X_train, X_test)

```overall_imp``` contains an ordered list of the features according to their importance. It's interesting to see that in our case the most significant value is ```M```, followed by ```F``` and then by ```R```.

In [ ]:
overall_imp